In [1]:
import os

In [2]:
# from loguru import logger
# from typing import Optional
# import requests
# import pandas as pd
# import json
# import matplotlib.pyplot as plt


# def get_sentiment_data(
#         ticker: str,
#         time_from: pd.Timestamp,
#         time_to: pd.Timestamp,
#         feed: Optional[list] = None,
# ) -> dict:
#     """Get sentiment data from the alpha vantage"""
#     # if feed is None, set it to an empty list
#     if feed is None:
#         feed = []

#     logger.info(f"Retrieving sentiment data from {time_from} to {time_to}")

#     api_key = os.environ['alphavantageapikey']
#     url = "https://www.alphavantage.co/query"
#     params = {
#         "function": "NEWS_SENTIMENT",
#         "tickers": ticker,
#         "time_from": time_from.strftime("%Y%m%dT%H%M"),
#         "time_to": time_to.strftime("%Y%m%dT%H%M"),
#         "apikey": api_key,
#         "limit": 200,
#     }
#     response = requests.get(url, params=params)
#     # logger.info(response.url)
#     result = response.json()
#     # need a check here in case there is no more data
#     if "feed" not in result or len(result["feed"]) == 0:
#         logger.warning("Empty feed returned, quitting")
#         return feed
#     # by default the result is starts with the newest and goes to the oldest
#     # as more data is collected, want to maintain that order
#     result_feed = result["feed"]
#     newest = pd.Timestamp(result_feed[0]["time_published"])
#     oldest = pd.Timestamp(result_feed[-1]["time_published"])
#     logger.info(f"newest date received {newest}")
#     logger.info(f"oldest date received {oldest}")
#     feed = feed + result["feed"]
#     # take a little bit of time away from oldest
#     oldest = oldest - pd.Timedelta("1T")
#     if oldest <= time_from:
#         logger.warning(f"{oldest:=} < time from, quitting")
#         return feed
#     logger.info(f"Fetching data from {time_from} to {oldest}")
#     return get_sentiment_data(ticker, time_from, oldest, feed=feed)


# bank_symbols = [
#     "JPM", "BAC", "WFC", "C", "USB", "TFC", "PNC", "BK", "STT", "COF", "TD", "SCHW", "MS", "GS", "FITB",
#     "CFG", "KEY", "RF", "MTB", "HBAN", "ALLY", "BBT", "STI", "ZION", "CMA", "NTRS", "PBCT", "SIVB", "BXS",
#     "SNV", "FRC", "WAL", "UMPQ", "FCNCA", "PACW", "FHN", "BKU", "EWBC", "IBKC", "ONB", "CIT", "PNFP",
#     "ASB", "CADE", "GWB", "NYCB", "BOH", "CFR", "FFIN", "TCBI", "HOMB", "PB", "UBSI", "WTFC", "FCFS",
#     "INDB", "GBCI", "HWC", "WSFS", "SASR", "COLB", "FMBI", "CVBF", "BMTC", "BOKF", "CATY", "FIBK", "SFNC",
#     "BKCC", "FULT", "FNB", "BANC", "BANR", "FBK", "FCBC", "FFBC", "GATX", "HOPE", "LBAI", "LTXB", "OZK",
#     "PEBO", "SFBS", "TCF", "TRMK", "UBCP", "UCBI", "VLY", "WABC", "WAFD", "WBS", "YDKN"
# ]

# for stock_ticker in bank_symbols:
#     logger.info(f"Running bank {stock_ticker}")
#     from_date = pd.Timestamp("2020-01-01")
#     to_date = pd.Timestamp.today()
#     feed = get_sentiment_data(stock_ticker, from_date, to_date)
#     with open(f"{stock_ticker}_sentiment.json", "w") as f:
#         json.dump(feed, f)

2023-04-09 15:57:46.303 | INFO     | __main__:<module>:67 - Running bank JPM
2023-04-09 15:57:46.304 | INFO     | __main__:get_sentiment_data:20 - Retrieving sentiment data from 2020-01-01 00:00:00 to 2023-04-09 15:57:46.304808
2023-04-09 15:57:47.017 | INFO     | __main__:get_sentiment_data:44 - newest date received 2023-04-09 14:01:00
2023-04-09 15:57:47.018 | INFO     | __main__:get_sentiment_data:45 - oldest date received 2023-04-05 06:40:50
2023-04-09 15:57:47.019 | INFO     | __main__:get_sentiment_data:52 - Fetching data from 2020-01-01 00:00:00 to 2023-04-05 06:39:50
2023-04-09 15:57:47.020 | INFO     | __main__:get_sentiment_data:20 - Retrieving sentiment data from 2020-01-01 00:00:00 to 2023-04-05 06:39:50
2023-04-09 15:57:47.359 | INFO     | __main__:get_sentiment_data:44 - newest date received 2023-04-05 05:23:50
2023-04-09 15:57:47.361 | INFO     | __main__:get_sentiment_data:45 - oldest date received 2023-03-31 14:45:29
2023-04-09 15:57:47.362 | INFO     | __main__:get_se

In [15]:
import numpy as np


def interpolate(series, new_index):
    new_dates = pd.Series(data=np.nan, index=new_index)
    series = pd.concat([series, new_dates], axis=0).sort_index()
    series = series.interpolate()
    # drop any duplicated indices
    series = series[~series.index.duplicated()]
    return series[new_index].ffill().bfill()


timestamps = pd.date_range(start="2022-05-01", end=pd.Timestamp.today(), freq="D")
sentiment_data = {}
required_len = 100
for stock_ticker in bank_symbols:
    with open(f"{stock_ticker}_sentiment.json", "r") as f:
        feed = json.load(f)
    if len(feed) == 0:
        logger.warning(f"{stock_ticker} has no data, skipping...")
        continue
    logger.info(f"Found data for {stock_ticker}")
    
    # extract the dates published
    dates = pd.to_datetime([x["time_published"] for x in feed])
    # extract the sentiment score for the ticket
    ticker_sentiment = []
    for article in feed:
        for article_ticker_sentiment in article["ticker_sentiment"]:
            if article_ticker_sentiment["ticker"] == stock_ticker:
                ticker_sentiment.append(article_ticker_sentiment)
    sentiment_score = [x["ticker_sentiment_score"] for x in ticker_sentiment]
    if len(dates) != len(sentiment_score):
        logger.error(f"Ticker {stock_ticker} has gone catastrophically wrong")

    sentiment = pd.Series(data=sentiment_score, index=dates)
    # Had to manually get SIVB data and it's not sorted. Need to sort before smoothing
    sentiment = sentiment.sort_index()
    if len(sentiment.index) < required_len:
        logger.warning(f"Ignoring {stock_ticker} as < {required_len} data points")
        continue
    sentiment = sentiment.rolling("7D").mean()
    interpolated = interpolate(sentiment, timestamps)
    interpolated.name = stock_ticker
    sentiment_data[stock_ticker] = interpolated

df = pd.concat(sentiment_data.values(), axis=1)
df = df[sorted(df.columns)]
df.to_csv("sentiment_data.csv")

2023-04-09 16:23:23.258 | INFO     | __main__:<module>:22 - Found data for JPM
2023-04-09 16:23:23.725 | INFO     | __main__:<module>:22 - Found data for BAC
2023-04-09 16:23:24.158 | INFO     | __main__:<module>:22 - Found data for WFC
2023-04-09 16:23:24.712 | INFO     | __main__:<module>:22 - Found data for C
2023-04-09 16:23:24.786 | WARNING  | __main__:<module>:20 - USB has no data, skipping...
2023-04-09 16:23:24.940 | INFO     | __main__:<module>:22 - Found data for TFC
2023-04-09 16:23:25.007 | INFO     | __main__:<module>:22 - Found data for PNC
2023-04-09 16:23:25.084 | INFO     | __main__:<module>:22 - Found data for BK
2023-04-09 16:23:25.299 | INFO     | __main__:<module>:22 - Found data for STT
2023-04-09 16:23:25.349 | INFO     | __main__:<module>:22 - Found data for COF
2023-04-09 16:23:25.385 | INFO     | __main__:<module>:22 - Found data for TD
2023-04-09 16:23:25.551 | INFO     | __main__:<module>:22 - Found data for SCHW
2023-04-09 16:23:26.306 | INFO     | __main__

Plot what's happening with sentiment scores for multiple banks over the last year

In [16]:
import plotly.graph_objects as go

ticker_to_name = {
    "JPM": "JPMorgan",
    "BAC": "Bank of America",
    "WFC": "Wells Fargo",
    "C": "Citigroup",
    "USB": "U.S. Bancorp",
    "TFC": "Truist Financial",
    "PNC": "PNC Financial",
    "BK": "Bank of New York Mellon",
    "STT": "State Street",
    "COF": "Capital One",
    "TD": "TD Group US",
    "SCHW": "Charles Schwab",
    "MS": "Morgan Stanley",
    "GS": "Goldman Sachs",
    "FITB": "Fifth Third Bancorp",
    "CFG": "Citizens Financial",
    "KEY": "KeyCorp",
    "RF": "Regions Financial",
    "MTB": "M&T Bank",
    "HBAN": "Huntington Bancshares",
    "ALLY": "Ally Financial",
    "BBT": "BB&T",
    "STI": "SunTrust Banks",
    "ZION": "Zions Bancorporation",
    "CMA": "Comerica",
    "NTRS": "Northern Trust",
    "PBCT": "People's United",
    "SIVB": "SVB Financial",
    "BXS": "BancorpSouth",
    "SNV": "Synovus Financial",
    "FRC": "First Republic",
    "WAL": "Western Alliance",
    "UMPQ": "Umpqua Holdings",
    "FCNCA": "First Citizens BancShares",
    "PACW": "PacWest Bancorp",
    "FHN": "First Horizon National",
    "BKU": "BankUnited",
    "EWBC": "East West Bancorp",
    "IBKC": "IBERIABANK",
    "ONB": "Old National",
    "CIT": "CIT Group",
    "PNFP": "Pinnacle Financial",
    "ASB": "Associated Banc",
    "CADE": "Cadence Bancorporation",
    "GWB": "Great Western Bancorp",
    "NYCB": "New York Community",
    "BOH": "Bank of Hawaii",
    "CFR": "Cullen/Frost Bankers",
    "FFIN": "First Financial Bankshares",
    "TCBI": "Texas Capital Bancshares",
    "HOMB": "Home BancShares",
    "PB": "Prosperity Bancshares",
    "UBSI": "United Bankshares",
    "WTFC": "Wintrust Financial",
    "FCFS": "FirstCash",
    "INDB": "Independent Bank",
    "GBCI": "Glacier Bancorp",
    "HWC": "Hancock Whitney",
    "WSFS": "WSFS Financial",
    "SASR": "Sandy Spring Bancorp",
    "COLB": "Columbia Banking",
    "FMBI": "First Midwest Bancorp",
    "CVBF": "CVB Financial",
    "BMTC": "Bryn Mawr Bank",
    "BOKF": "BOK Financial",
    "CATY": "Cathay General Bancorp",
    "FIBK": "First Interstate BancSystem",
    "SFNC": "Simmons First National",
    "BKCC": "BlackRock Capital",
    "FULT": "Fulton Financial",
    "FNB": "F.N.B.",
    "BANC": "Banc of California",
    "BANR": "Banner",
    "FBK": "FB Financial",
    "FCBC": "First Community Bancshares",
    "FFBC": "First Financial Bancorp",
    "GATX": "GATX",
    "HOPE": "Hope Bancorp",
    "LBAI": "Lakeland Bancorp",
    "LTXB": "LegacyTexas Financial",
    "OZK": "Bank OZK",
    "PEBO": "Peoples Bancorp",
    "SFBS": "ServisFirst Bancshares",
    "TCF": "TCF Financial",
    "TRMK": "Trustmark",
    "UBCP": "United Bancorp",
    "UCBI": "United Community Banks",
    "VLY": "Valley National Bancorp",
    "WABC": "Westamerica Bancorporation",
    "WAFD": "Washington Federal",
    "WBS": "Webster Financial",
    "YDKN": "Yadkin Financial",
}

# calculate out mean sentiment across all banks
mean_sentiment = df.mean(axis=1)

to_plot = df.columns
fig = go.Figure()
opacity = 0.1
fig.add_hrect(y0=-0.60, y1=-0.35, opacity=opacity, annotation={"text": "Bearish"}, annotation_position="bottom right",
              line_color=None, fillcolor="#FF6900")
fig.add_hrect(y0=-0.35, y1=-0.15, opacity=opacity, annotation={"text": "Somewhat Bearish"},
              annotation_position="bottom right", line_color=None, fillcolor="#FCB900")
fig.add_hrect(y0=-0.15, y1=0.15, opacity=opacity, annotation={"text": "Neutral"}, annotation_position="bottom right",
              line_color=None, fillcolor="#D9E3F0")
fig.add_hrect(y0=0.15, y1=0.35, opacity=opacity, annotation={"text": "Somewhat Bullish"},
              annotation_position="bottom right", line_color=None, fillcolor="#0693E3")
fig.add_hrect(y0=0.35, y1=0.80, opacity=opacity, annotation={"text": "Bullish"}, annotation_position="bottom right",
              line_color=None, fillcolor="#00D084")
for stock_ticker in to_plot:
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[stock_ticker],
            mode='lines',
            name=ticker_to_name[stock_ticker],
            meta=[ticker_to_name[stock_ticker]],
            hovertemplate='<br>'.join([
                'Bank: %{meta[0]}',
                'X: %{x}',
                'Y: %{y}'
            ])
        )
    )
fig.add_trace(
    go.Scatter(
        x=mean_sentiment.index,
        y=mean_sentiment.values,
        mode='lines',
        name="Mean sentiment",
        meta=["Mean sentiment"],
        hovertemplate='<br>'.join([
            'Bank: %{meta[0]}',
            'X: %{x}',
            'Y: %{y}'
        ]),
        line={"width": 3, "color": "black"}
    )
)
fig.update_layout(height=800)
fig.show()

Let's remove the mean trend for everything

In [17]:
fig = go.Figure()
for stock_ticker in to_plot:
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[stock_ticker] - mean_sentiment,
            mode='lines',
            name=ticker_to_name[stock_ticker],
            meta=[ticker_to_name[stock_ticker]],
            hovertemplate='<br>'.join([
                'Bank: %{meta[0]}',
                'X: %{x}',
                'Y: %{y}'
            ])
        )
    )
fig.update_layout(height=800)
fig.show()

These all look extremely correlated. Let's try and make a correlation matrix with them and see how it looks.

In [18]:
corr = df.corr()
corr = corr.sort_index()
# corr = corr[sorted(corr.columns)]
fig = go.Figure()
heatmap = fig.add_trace(
    go.Heatmap(
        z=corr.values,
        x=corr.index.values,
        y=corr.columns.values,
        zmin=0,
        zmax=1
    )
)
fig.update_yaxes(autorange="reversed")
fig.update_layout(height=1_000)
# Show the plot
fig.show()

Okay, this feels a little confusing, I did expect sentiment not to be as strong, especially as we're heading into what looks like it could be a financial crash. So why are sentiment scores somewhat bullish right now? Let's try plotting with some macro economic factors

In [19]:
api_key = os.environ['alphavantageapikey']

def get_economic_indicators():
    indicators = [
        'REAL_GDP',
        'REAL_GDP_PER_CAPITA',
        'TREASURY_YIELD',
        'FEDERAL_FUNDS_RATE',
        'CPI',
        'INFLATION',
        'RETAIL_SALES',
        'DURABLES',
        'UNEMPLOYMENT',
        'NONFARM_PAYROLL',
    ]

    macro_data = {}
    for indicator in indicators:
        symbol = indicator
        indicator_url = f"https://www.alphavantage.co/query?function={symbol}&apikey={api_key}&interval=quarterly"
        indicator_response = requests.get(indicator_url)

        if indicator_response.status_code != 200:
            logger.warning(f"Error for {symbol}: {indicator_response.status_code} {indicator_response.reason}")
            continue

        indicator_data = indicator_response.json()
        if 'Error Message' in indicator_data:
            logger.warning(f"Error for {symbol}: {indicator_data['Error Message']}")
            continue

        logger.info(f"Got data for indicator {indicator}")
        indicator_df = pd.DataFrame.from_records(indicator_data["data"])
        indicator_df["value"] = pd.to_numeric(indicator_df["value"])
        indicator_df["date"] = pd.to_datetime(indicator_df["date"])
        indicator_df = indicator_df.set_index("date")
        indicator_series = indicator_df["value"]
        indicator_series.name = indicator_data["name"]
        indicator_series = indicator_series.sort_index()
        macro_data[indicator_data["name"]] = indicator_series
    return macro_data

macro_indicators = get_economic_indicators()


2023-04-09 16:23:29.700 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator REAL_GDP
2023-04-09 16:23:30.013 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator REAL_GDP_PER_CAPITA
2023-04-09 16:23:30.411 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator TREASURY_YIELD
2023-04-09 16:23:30.809 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator FEDERAL_FUNDS_RATE
2023-04-09 16:23:31.657 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator CPI
2023-04-09 16:23:31.884 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator INFLATION
2023-04-09 16:23:32.214 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator RETAIL_SALES
2023-04-09 16:23:32.534 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator DURABLES
2023-04-09 16:23:33.117 | INFO     | __main__:get_economic_indicators:32 - Got data for indicator UNEMPLOYMENT
2023-0

Plot the mean sentiment data with macro indicators. For now, just normalise everything so you can see the trends on a single axis. 

In [20]:
fig = go.Figure()
for macro_indicator, indicator_series in macro_indicators.items():
    normed_indicator = (indicator_series - indicator_series.mean())/(indicator_series.max() - indicator_series.min())
    normed_indicator = normed_indicator["2022-01-01T00:00:00":]
    fig.add_trace(
        go.Scatter(
            x=normed_indicator.index,
            y=normed_indicator.values,
            mode='lines',
            name=macro_indicator,
            meta=[macro_indicator],
            hovertemplate='<br>'.join([
                'Bank: %{meta[0]}',
                'X: %{x}',
                'Y: %{y}'
            ])
        )
    )
fig.add_trace(
    go.Scatter(
        x=mean_sentiment.index,
        y=mean_sentiment.values,
        mode='lines',
        name="Mean sentiment",
        meta=["Mean sentiment"],
        hovertemplate='<br>'.join([
            'Bank: %{meta[0]}',
            'X: %{x}',
            'Y: %{y}'
        ]),
        line={"width": 3, "color": "black"}
    )
)
fig.update_layout(height=800)
fig.show()

In [21]:
bank_symbols = [
    "JPM",  # JPMorgan Chase & Co.
    "BAC",  # Bank of America Corp.
    "WFC",  # Wells Fargo & Co.
    "C",    # Citigroup Inc.
    "USB",  # U.S. Bancorp
    "TFC",  # Truist Financial Corporation
    "PNC",  # PNC Financial Services Group Inc.
    "BK",   # Bank of New York Mellon Corp.
    "STT",  # State Street Corp.
    "COF",  # Capital One Financial Corp.
    "TD",   # TD Group US Holdings LLC
    "SCHW", # Charles Schwab Corporation
    "MS",   # Morgan Stanley
    "GS",   # Goldman Sachs Group Inc.
    "FITB", # Fifth Third Bancorp
    "CFG",  # Citizens Financial Group Inc.
    "KEY",  # KeyCorp
    "RF",   # Regions Financial Corp.
    "MTB",  # M&T Bank Corp.
    "HBAN", # Huntington Bancshares Inc.
    "ALLY", # Ally Financial Inc.
    "BBT",  # BB&T Corp.
    "STI",  # SunTrust Banks Inc.
    "ZION", # Zions Bancorporation, N.A.
    "CMA",  # Comerica Incorporated
    "NTRS", # Northern Trust Corporation
    "PBCT", # People's United Financial, Inc.
    "SIVB", # SVB Financial Group
    "BXS",  # BancorpSouth Bank
    "SNV",  # Synovus Financial Corp.
    # "FRC",  # First Republic Bank
    "WAL",  # Western Alliance Bancorporation
    "UMPQ", # Umpqua Holdings Corporation
    "FCNCA",# First Citizens BancShares, Inc.
    "PACW", # PacWest Bancorp
    "FHN",  # First Horizon National Corp.
    "BKU",  # BankUnited, Inc.
    "EWBC", # East West Bancorp, Inc.
    "IBKC", # IBERIABANK Corporation
    "ONB",  # Old National Bancorp
    "CIT",  # CIT Group Inc.
    "PNFP", # Pinnacle Financial Partners, Inc.
    "ASB",  # Associated Banc-Corp
    "CADE", # Cadence Bancorporation
    "GWB",  # Great Western Bancorp, Inc.
    "NYCB", # New York Community Bancorp, Inc.
    "BOH",  # Bank of Hawaii Corporation
    "CFR",  # Cullen/Frost Bankers, Inc.
    "FFIN", # First Financial Bankshares, Inc.
    "TCBI", # Texas Capital Bancshares, Inc.
    "HOMB", # Home BancShares, Inc.
    "PB",   # Prosperity Bancshares, Inc.
    "UBSI", # United Bankshares, Inc.
    "WTFC", # Wintrust Financial Corporation
    # "FCFS", # FirstCash, Inc.
    "INDB", # Independent Bank Corp.
    "GBCI", # Glacier Bancorp, Inc.
    "HWC",  # Hancock Whitney Corporation
    "WSFS", # WSFS Financial Corporation
    "SASR", # Sandy Spring Bancorp, Inc.
    "COLB", # Columbia Banking System, Inc.
    "FMBI", # First Midwest Bancorp, Inc.
    "CVBF", # CVB Financial Corp.
    "BMTC", # Bryn Mawr Bank Corporation
    "BOKF", # BOK Financial Corporation
    "CATY", # Cathay General Bancorp
    "FIBK", # First Interstate BancSystem, Inc.
    "SFNC", # Simmons First National Corporation
    # "BKCC", # BlackRock Capital Investment Corporation
    "FULT", # Fulton Financial Corporation
    "FNB",  # F.N.B. Corporation
    "BANC", # Banc of California, Inc.
    "BANR", # Banner Corporation
    "FBK",  # FB Financial Corporation
    "FCBC", # First Community Bancshares, Inc.
    "FFBC", # First Financial Bancorp.
    "GATX", # GATX Corporation
    "HOPE", # Hope Bancorp, Inc.
    "LBAI", # Lakeland Bancorp, Inc.
    "LTXB", # LegacyTexas Financial Group, Inc.
    "OZK",  # Bank OZK
    "PEBO", # Peoples Bancorp Inc.
    "SFBS", # ServisFirst Bancshares, Inc.
    "TCF",  # TCF Financial Corporation
    "TRMK", # Trustmark Corporation
    "UBCP", # United Bancorp, Inc.
    "UCBI", # United Community Banks, Inc.
    "VLY",  # Valley National Bancorp
    "WABC", # Westamerica Bancorporation
    "WAFD", # Washington Federal, Inc.
    "WBS",  # Webster Financial Corporation
    "YDKN", # Yadkin Financial Corporation
]


In [22]:
bank_name_dict = {
    "JPM": "JPMorgan",
    "BAC": "Bank of America",
    "WFC": "Wells Fargo",
    "C": "Citigroup",
    "USB": "U.S. Bancorp",
    "TFC": "Truist Financial",
    "PNC": "PNC Financial",
    "BK": "Bank of New York Mellon",
    "STT": "State Street",
    "COF": "Capital One",
    "TD": "TD Group US",
    "SCHW": "Charles Schwab",
    "MS": "Morgan Stanley",
    "GS": "Goldman Sachs",
    "FITB": "Fifth Third Bancorp",
    "CFG": "Citizens Financial",
    "KEY": "KeyCorp",
    "RF": "Regions Financial",
    "MTB": "M&T Bank",
    "HBAN": "Huntington Bancshares",
    "ALLY": "Ally Financial",
    "BBT": "BB&T",
    "STI": "SunTrust Banks",
    "ZION": "Zions Bancorporation",
    "CMA": "Comerica",
    "NTRS": "Northern Trust",
    "PBCT": "People's United",
    "SIVB": "SVB Financial",
    "BXS": "BancorpSouth",
    "SNV": "Synovus Financial",
    "FRC": "First Republic",
    "WAL": "Western Alliance",
    "UMPQ": "Umpqua Holdings",
    "FCNCA": "First Citizens BancShares",
    "PACW": "PacWest Bancorp",
    "FHN": "First Horizon National",
    "BKU": "BankUnited",
    "EWBC": "East West Bancorp",
    "IBKC": "IBERIABANK",
    "ONB": "Old National",
    "CIT": "CIT Group",
    "PNFP": "Pinnacle Financial",
    "ASB": "Associated Banc",
    "CADE": "Cadence Bancorporation",
    "GWB": "Great Western Bancorp",
    "NYCB": "New York Community",
    "BOH": "Bank of Hawaii",
    "CFR": "Cullen/Frost Bankers",
    "FFIN": "First Financial Bankshares",
    "TCBI": "Texas Capital Bancshares",
    "HOMB": "Home BancShares",
    "PB": "Prosperity Bancshares",
    "UBSI": "United Bankshares",
    "WTFC": "Wintrust Financial",
    "FCFS": "FirstCash",
    "INDB": "Independent Bank",
    "GBCI": "Glacier Bancorp",
    "HWC": "Hancock Whitney",
    "WSFS": "WSFS Financial",
    "SASR": "Sandy Spring Bancorp",
    "COLB": "Columbia Banking",
    "FMBI": "First Midwest Bancorp",
    "CVBF": "CVB Financial",
    "BMTC": "Bryn Mawr Bank",
    "BOKF": "BOK Financial",
    "CATY": "Cathay General Bancorp",
    "FIBK": "First Interstate BancSystem",
    "SFNC": "Simmons First National",
    "BKCC": "BlackRock Capital",
    "FULT": "Fulton Financial",
    "FNB": "F.N.B.",
    "BANC": "Banc of California",
    "BANR": "Banner",
    "FBK": "FB Financial",
    "FCBC": "First Community Bancshares",
    "FFBC": "First Financial Bancorp",
    "GATX": "GATX",
    "HOPE": "Hope Bancorp",
    "LBAI": "Lakeland Bancorp",
    "LTXB": "LegacyTexas Financial",
    "OZK": "Bank OZK",
    "PEBO": "Peoples Bancorp",
    "SFBS": "ServisFirst Bancshares",
    "TCF": "TCF Financial",
    "TRMK": "Trustmark",
    "UBCP": "United Bancorp",
    "UCBI": "United Community Banks",
    "VLY": "Valley National Bancorp",
    "WABC": "Westamerica Bancorporation",
    "WAFD": "Washington Federal",
    "WBS": "Webster Financial",
    "YDKN": "Yadkin Financial",
}

In [23]:
import pandas as pd
import json

In [24]:
# Define a function to convert a string to a float, or return None if the string is 'None'
def convert_to_float(value):
    if value == 'None':
        return None
    else:
        return float(value)

In [25]:
df_list = []
for bank_symbol in bank_symbols:
    file_path = f"/data/workspace_files/cash_flow/{bank_symbol}_CASH_FLOW.json"
    try:
        # Read JSON file
        with open(file_path, 'r') as file:
            json_data = json.load(file)
        df = pd.DataFrame.from_dict(json_data['quarterlyReports'])
        df['bank_symbol'] = bank_symbol
        df_list.append(df)
    except:
        continue
cash_flow_data = pd.concat(df_list)
ignore_columns = ['fiscalDateEnding', 'reportedCurrency', 'bank_symbol']
cash_flow_data.loc[:, ~cash_flow_data.columns.isin(ignore_columns)] = \
    cash_flow_data.loc[:, ~cash_flow_data.columns.isin(ignore_columns)].applymap(convert_to_float)
# cash_flow_data.applymap(convert_to_float)
cash_flow_data = cash_flow_data.astype(
    {
        'fiscalDateEnding': 'datetime64', 
        'reportedCurrency': 'object', 
        'operatingCashflow': 'float',
        'paymentsForOperatingActivities': 'float', 
        'proceedsFromOperatingActivities': 'float',
        'changeInOperatingLiabilities': 'float', 
        'changeInOperatingAssets': 'float',
        'depreciationDepletionAndAmortization': 'float', 
        'capitalExpenditures': 'float',
        'changeInReceivables': 'float', 
        'changeInInventory': 'float', 
        'profitLoss': 'float',
        'cashflowFromInvestment': 'float', 
        'cashflowFromFinancing': 'float',
        'proceedsFromRepaymentsOfShortTermDebt': 'float',
        'paymentsForRepurchaseOfCommonStock': 'float', 
        'paymentsForRepurchaseOfEquity': 'float',
        'paymentsForRepurchaseOfPreferredStock': 'float', 
        'dividendPayout': 'float',
        'dividendPayoutCommonStock': 'float', 
        'dividendPayoutPreferredStock': 'float',
        'proceedsFromIssuanceOfCommonStock': 'float',
        'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet': 'float',
        'proceedsFromIssuanceOfPreferredStock': 'float',
        'proceedsFromRepurchaseOfEquity': 'float', 
        'proceedsFromSaleOfTreasuryStock': 'float',
        'changeInCashAndCashEquivalents': 'float', 
        'changeInExchangeRate': 'float', 
        'netIncome': 'float',
    }
, errors='raise')

In [26]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Assume the cash_flow_data DataFrame already contains data for multiple banks with a 'bank_symbol' column

# Calculate key ratios and growth rates for each bank
cash_flow_data['operating_cash_flow_margin'] = cash_flow_data['operatingCashflow'] / cash_flow_data['profitLoss']
cash_flow_data['investment_cash_flow_margin'] = cash_flow_data['cashflowFromInvestment'] / cash_flow_data['profitLoss']
cash_flow_data['financing_cash_flow_margin'] = cash_flow_data['cashflowFromFinancing'] / cash_flow_data['profitLoss']

cash_flow_data['operating_cashflow_growth'] = cash_flow_data.groupby('bank_symbol')['operatingCashflow'].pct_change()
cash_flow_data['cashflow_from_investment_growth'] = cash_flow_data.groupby('bank_symbol')['cashflowFromInvestment'].pct_change()
cash_flow_data['cashflow_from_financing_growth'] = cash_flow_data.groupby('bank_symbol')['cashflowFromFinancing'].pct_change()
cash_flow_data['net_income_growth'] = cash_flow_data.groupby('bank_symbol')['netIncome'].pct_change()

# Get a list of unique bank symbols
unique_bank_symbols = sorted(cash_flow_data['bank_symbol'].unique())

# Create separate plots for each key cash flow component and ratio
titles = ['Operating Cash Flow Margin', 'Investment Cash Flow Margin', 'Financing Cash Flow Margin', 'Net Income Growth']
data_columns = ['operating_cash_flow_margin', 'investment_cash_flow_margin', 'financing_cash_flow_margin', 'net_income_growth']

for title, data_column in zip(titles, data_columns):
    fig = go.Figure()

    for bank_symbol in unique_bank_symbols:
        bank_data = cash_flow_data[cash_flow_data['bank_symbol'] == bank_symbol]
        visible = 'legendonly' if bank_symbol != 'SIVB' else True
        fig.add_trace(go.Scatter(x=bank_data['fiscalDateEnding'], y=bank_data[data_column], mode='lines+markers', name=f'{bank_symbol} {title}', visible=visible))

    # Calculate the mean for the current data_column across all banks
    mean_data = cash_flow_data.groupby('fiscalDateEnding')[data_column].mean().interpolate().reset_index()
    fig.add_trace(go.Scatter(x=mean_data['fiscalDateEnding'], y=mean_data[data_column], mode='lines+markers', name=f'Mean {title}', line=dict(color='black', dash='dot'), visible=True))

    fig.update_layout(title=title)
    fig.show()

# Create a plot for the cash balance trend for all banks
fig2 = go.Figure()

for bank_symbol in unique_bank_symbols:
    bank_data = cash_flow_data[cash_flow_data['bank_symbol'] == bank_symbol]
    visible = 'legendonly' if bank_symbol != 'SIVB' else True
    fig2.add_trace(go.Scatter(x=bank_data['fiscalDateEnding'], y=bank_data['changeInCashAndCashEquivalents'], mode='lines+markers', name=f'{bank_symbol} Change in Cash and Cash Equivalents', visible=visible))

# Calculate the mean change in cash and cash equivalents across all banks
mean_data = cash_flow_data.groupby('fiscalDateEnding')['changeInCashAndCashEquivalents'].mean().interpolate().reset_index()
fig2.add_trace(go.Scatter(x=mean_data['fiscalDateEnding'], y=mean_data['changeInCashAndCashEquivalents'], mode='lines+markers', name='Mean Change in Cash and Cash Equivalents', line=dict(color='black', dash='dot'), visible=True))

fig2.update_layout(title='Trend in Cash Balance for All Banks')

# Show the cash balance trend plot
fig2.show()